### ECCITE_seq Dataset Download and Preprocessing 

Preprocessing data code from repository suinleelab **contrastiveVI**: https://github.com/suinleelab/contrastiveVI/blob/main/contrastive_vi/data/datasets/papalexi_2021.py

The link to download the data is in the code. The ECCITE_seq data is from Gene Expression Omnibus (GEO) public genomics data repository (https://www.ncbi.nlm.nih.gov/geo/info/download.html) The license of the organization can be found here: https://catalog.data.gov/dataset/gene-expression-omnibus-geo. 

The python **anndata** package is found here: https://github.com/scverse/anndata?tab=BSD-3-Clause-1-ov-file

The **scanpy** package is found here: https://github.com/scverse/scanpy

The **requests** package is found here: https://github.com/psf/requests

In [4]:
"""
Download, read, and preprocess Papalexi et al. (2021) expression data.

Single-cell expression data from Papalexi et al. Characterizing the molecular regulation
of inhibitory immune checkpoints with multimodal single-cell screens. (Nature Genetics
2021)
"""
import os
import shutil

import pandas as pd
import scanpy as sc
from anndata import AnnData

import requests


def download_binary_file(file_url: str, output_path: str) -> None:
    """
    Download binary data file from a URL.

    Args:
    ----
        file_url: URL where the file is hosted.
        output_path: Output path for the downloaded file.

    Returns
    -------
        None.
    """
    request = requests.get(file_url)
    with open(output_path, "wb") as f:
        f.write(request.content)
    print(f"Downloaded data from {file_url} at {output_path}")



def download_papalexi_2021(output_path: str) -> None:
    """
    Download Papalexi et al. 2021 data from the hosting URLs.

    Args:
    ----
        output_path: Output path to store the downloaded and unzipped
        directories.

    Returns
    -------
        None. File directories are downloaded to output_path.
    """

    counts_data_url = (
        "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE153056&format=file"
    )
    data_output_filename = os.path.join(output_path, "GSE153056_RAW.tar")
    download_binary_file(counts_data_url, data_output_filename)
    shutil.unpack_archive(data_output_filename, output_path)

    metadata_url = (
        "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE153056&"
        "format=file&file=GSE153056_ECCITE_metadata.tsv.gz"
    )
    metadata_filename = os.path.join(output_path, metadata_url.split("=")[-1])
    download_binary_file(metadata_url, metadata_filename)


def read_papalexi_2021(file_directory: str) -> pd.DataFrame:
    """
    Read the expression data for Papalexi et al. 2021 in the given directory.

    Args:
    ----
        file_directory: Directory containing Papalexi et al. 2021 data.

    Returns
    -------
        A pandas dataframe, with each column representing a cell
        and each row representing a gene feature.
    """

    matrix = pd.read_csv(
        os.path.join(file_directory, "GSM4633614_ECCITE_cDNA_counts.tsv.gz"),
        sep="\t",
        index_col=0,
    )
    return matrix


def preprocess_papalexi_2021(download_path: str, n_top_genes: int) -> AnnData:
    """
    Preprocess expression data from Papalexi et al. 2021.

    Args:
    ----
        download_path: Path containing the downloaded Papalexi et al. 2021 data files.
        n_top_genes: Number of most variable genes to retain.

    Returns
    -------
        An AnnData object containing single-cell expression data. The layer
        "count" contains the count data for the most variable genes. The .X
        variable contains the normalized and log-transformed data for the most variable
        genes. A copy of data with all genes is stored in .raw.
    """

    df = read_papalexi_2021(download_path)

    # Switch dataframe from gene rows and cell columns to cell rows and gene columns
    df = df.transpose()

    metadata = pd.read_csv(
        os.path.join(download_path, "GSE153056_ECCITE_metadata.tsv.gz"),
        sep="\t",
        index_col=0,
    )

    metadata.to_csv("eccite_metadata.csv")

    # Note: By initializing the anndata object from a dataframe, variable names
    # are automatically stored in adata.var
    adata = AnnData(df)
    adata.obs = metadata

    # Protein measurements also collected as part of CITE-Seq
    protein_counts_df = pd.read_csv(
        os.path.join(download_path, "GSM4633615_ECCITE_ADT_counts.tsv.gz"),
        sep="\t",
        index_col=0,
    )

    # Switch dataframe from protein rows and cell columns to cell rows and protein
    # columns
    protein_counts_df = protein_counts_df.transpose()

    # Storing protein counts in an obsm field as expected by totalVI
    # (see https://docs.scvi-tools.org/en/stable/tutorials/notebooks/totalVI.html
    # for an example). Since `protein_counts_df` is annotated with protein names,
    # our obsm field will retain them as well.
    # adata.obsm[constants.PROTEIN_EXPRESSION_KEY] = protein_counts_df

    adata.layers["count"] = adata.X.copy()
    sc.pp.normalize_total(adata)
    sc.pp.log1p(adata)
    adata.raw = adata
    sc.pp.highly_variable_genes(
        adata, flavor="seurat_v3", n_top_genes=n_top_genes, layer="count", subset=True
    )
    adata = adata[adata.layers["count"].sum(1) != 0]  # Remove cells with all zeros.
    return adata

In [1]:
root_data_path = "..."   # local computer data path to downloaad the dataset. 

download_papalexi_2021(root_data_path)

In [6]:
n_top_genes = 1000 

data = preprocess_papalexi_2021(root_data_path, n_top_genes)

### Process to Get Foreground and Background  dataset
Foreground is treatment, and background is control

In [11]:
control = data[data.obs["crispr"] == "NT"]
m, k = control.shape
print(m)
print(control.obs)

2386
                    orig.ident  nCount_RNA  nFeature_RNA  nCount_HTO  \
l1_AAACGGGAGACAGAGA      Lane1       14941          3918          97   
l1_AAAGCAACAGTGGAGT      Lane1        6592          2564          41   
l1_AAAGTAGAGGATCGCA      Lane1        9783          3334          90   
l1_AACGTTGTCATGCAAC      Lane1       12147          3233          31   
l1_AAGGAGCCATTAGGCT      Lane1       10656          3259          91   
...                        ...         ...           ...         ...   
l8_TTTCCTCTCAGCAACT      Lane8       16790          4298         116   
l8_TTTGGTTGTTCCTCCA      Lane8        5093          1756         133   
l8_TTTGGTTTCGACCAGC      Lane8        5837          2302          87   
l8_TTTGTCAAGTTACGGG      Lane8       22441          4624         170   
l8_TTTGTCAGTGACGGTA      Lane8       17553          3787         528   

                     nFeature_HTO  nCount_GDO  nFeature_GDO  nCount_ADT  \
l1_AAACGGGAGACAGAGA             5         124          

NT stands for Non-targeting

In [12]:
treatment = data[data.obs["crispr"] != "NT"]
t, k = treatment.shape
print(t)
print(treatment.obs)

18343
                    orig.ident  nCount_RNA  nFeature_RNA  nCount_HTO  \
l1_AAACCTGAGCCAGAAC      Lane1       17207          3942          99   
l1_AAACCTGAGTGGACGT      Lane1        9506          2948          35   
l1_AAACCTGCATGAGCGA      Lane1       15256          4258          66   
l1_AAACCTGTCTTGTCAT      Lane1        5135          1780          22   
l1_AAACGGGAGAACAACT      Lane1        9673          2671          99   
...                        ...         ...           ...         ...   
l8_TTTGTCAGTAGGCATG      Lane8        9446          3240         111   
l8_TTTGTCAGTCACTTCC      Lane8       20863          4884         294   
l8_TTTGTCAGTTCCACAA      Lane8       15106          4185         154   
l8_TTTGTCATCACGCATA      Lane8       11209          3204         132   
l8_TTTGTCATCTGTACGA      Lane8       23088          5073          95   

                     nFeature_HTO  nCount_GDO  nFeature_GDO  nCount_ADT  \
l1_AAACCTGAGCCAGAAC             4         576         

In [13]:
background = control.to_df()
foreground = treatment.to_df()

background.to_csv("ECCITE_seq_back.csv")
foreground.to_csv("ECCITE_seq_fore.csv")